### 0. Import Libs

In [1]:
# kernel must be msd2
import sys;sys.path.append('../')
import constant as c
import os
import pandas as pd
import warnings; warnings.filterwarnings("ignore")
from util.epsg_transform import *
from shapely.geometry import *

### 1. Get Metro Data

In [2]:
# get metro origin/destination df
metro_folder_path = os.path.join(c.SSD_PATH,'metro')
metro_fname = '서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv'
metro_path = os.path.join(metro_folder_path, metro_fname)
metro = pd.read_csv(metro_path, encoding = 'euc-kr')

In [3]:
metro

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202303,1호선,동대문,775,8,11503,2638,10255,7250,15728,...,8898,588,2612,1,247,0,0,0,0,20230403
1,202303,1호선,동묘앞,213,2,2916,1024,3653,5289,7749,...,4133,99,1587,1,299,0,0,0,0,20230403
2,202303,1호선,서울역,677,35,8460,8722,13265,53198,43763,...,14624,2352,3091,0,220,0,0,0,0,20230403
3,202303,1호선,시청,113,0,2396,4586,4034,24139,8529,...,4654,362,1076,2,27,0,1,0,0,20230403
4,202303,1호선,신설동,460,11,9074,2309,9800,8642,22348,...,9422,384,2110,0,5,1,2,0,0,20230403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58176,201501,중앙선,운길산,0,0,177,56,642,403,1292,...,633,3,222,0,0,0,0,0,0,20151223
58177,201501,중앙선,원덕,1,0,292,4,415,46,537,...,383,3,157,0,0,0,0,0,0,20151223
58178,201501,중앙선,중랑,14,1,4929,424,8539,2084,22160,...,6501,86,1617,0,0,0,0,0,0,20151223
58179,201501,중앙선,팔당,0,0,161,50,368,149,1227,...,336,8,85,0,0,0,0,0,0,20151223


In [4]:
yyyymms = ['201801', '201901', '202001', '202101', '202201']
metro = metro[metro['사용월'].isin([201801, 201901, 202001, 202101, 202201])]
metro = metro.drop(columns='작업일자')
metro['지하철역'] = metro['지하철역'].str.replace(r'\s*\([^)]*\)', '')

# get metro location df
metroloc_fname = 'metroloc.csv'
metroloc_path = os.path.join(metro_folder_path, metroloc_fname)
metroloc = pd.read_csv(metroloc_path, encoding = 'utf-8')
metroloc['name'] = metroloc['name'].apply(lambda x:x.strip('"'))
metroloc['line'] = metroloc['line'].apply(lambda x:x.strip('"'))
metroloc['line'] = metroloc['line'].str.lstrip('0')
metroloc = metroloc.rename(columns = {'lat': 'y', 'lng': 'x', 'line': '호선명', 'name': '지하철역'})
metroloc['x'] = pd.to_numeric(metroloc['x'], errors='coerce')
metroloc['y'] = pd.to_numeric(metroloc['y'], errors='coerce')

# merge two dfs
df_metro = pd.merge(metro, metroloc, on = ['호선명', '지하철역'])
df_metro['사용월'] = df_metro['사용월'].astype(str)
df_metro['year'] = df_metro['사용월'].str[:4]


x_5179, y_5179 = coord_tranformation(list(df_metro.x), list(df_metro.y), "epsg:4326", "epsg:5179")
df_metro['x_5179'] = x_5179
df_metro['y_5179'] = y_5179

df_metro = df_metro.rename(columns=c.DF_METRO_COL_AMEND_DICT)
df_metro.to_csv('../asset/preprocess/features/df_metro.csv')

In [5]:
# Read df_metro
df_metro = pd.read_csv('../asset/preprocess/features/df_metro.csv', index_col = 0)